In [1]:
%load_ext autoreload
%autoreload 2
from TNR import *

In [10]:
chi={}
chi['w'],chi['y'],chi['u'],chi['v']=6,8,6,6
chiw,chiy,chiu,chiv=chi['w'],chi['y'],chi['u'],chi['v']

J=1
# htemp=(KP(s0,sx,sx,s0)+KP(s0,sz,sz,s0))+\
#     J*(KP(sx,sx,s0,s0)+KP(sz,sz,s0,s0)+KP(s0,s0,sx,sx)+KP(s0,s0,sz,sz))/2
htemp=KP(sz,sz)+J*(KP(sx,s0)+KP(s0,sx))/2
# htemp=(KP(sz,s0)+KP(s0,sz))/2+(KP(sx,s0)+KP(s0,sx))/2
# htemp=KP(sx,sx)+KP(sz,sz)
# htemp=-KP(R,R.conj())-J*(KP(M,np.eye(3))+KP(np.eye(3),M))/2
# htemp=-KP(R,R.conj())-KP(M,M)
# htemp+=htemp.T.conj()

d=int(np.sqrt(htemp.shape[0]))
Ainit=expm(-0.002*htemp).reshape(d,d,d,d)
A0=Asplit(Ainit,chi)
for i in range(10):
    A0=coarsen(A0,chi)

# classical ising:
# na=np.newaxis
# s=np.array([1,-1])
# Tc=2/np.log(1+np.sqrt(2))
# T=1*Tc
# A0=np.exp(-1/T*(s[:,na,na,na]*s[na,:,na,na]+s[na,:,na,na]*s[na,na,:,na]\
#                 +s[na,na,:,na]*s[na,na,na,:]+s[na,na,na,:]*s[:,na,na,na]))
# A0/=np.linalg.norm(A0)

vertical truncation err: 0.0
horizontal truncation err: 1.3307111039703433e-12
Aout shape: (3, 4, 3, 4)
Coarsen truncation error: 6.63999e-11
Coarsen truncation error: 5.52341e-13
Coarsen truncation error: 2.07825e-11
Coarsen truncation error: 2.11725e-11
Coarsen truncation error: 4.44935e-12
Coarsen truncation error: 8.79814e-11
Coarsen truncation error: 2.3348e-08
Coarsen truncation error: 3.26661e-06
Coarsen truncation error: 0.000170188
Coarsen truncation error: 0.00372825


In [11]:
RGsteps=5
A_list=[None]*(RGsteps+1)
Anorm_list=[None]*(RGsteps+1)
B_list,U_list,vl_list,vr_list,yl_list,yr_list,D_list,w_list,u_list,v_list=[[None]*RGsteps for i in range(10)]
A_list[0]=A0

for i in range(RGsteps):
    print('RG step: ',i)
    Aold=A_list[i]
    B,U,vl,vr,yl,yr,D,w,u,v,A,Anorm=doTNR(Aold,chi)
    B_list[i],U_list[i],vl_list[i],vr_list[i],yl_list[i],yr_list[i],D_list[i],w_list[i],u_list[i],v_list[i],A_list[i+1],Anorm_list[i+1]=\
    B,U,vl,vr,yl,yr,D,w,u,v,A,Anorm
    print('A.shape= ',A.shape)
    print()

RG step:  0
iter=0, U.Uenv=0.393018
iter=100, U.Uenv=0.393249
iter=200, U.Uenv=0.393249
error in U, vl, vr optimization: 0.0012854
error in yl, yr, D optimization: 0.010169
error in w optimization: 0.0079128
iter=0, u.uenv=0.863797, v.venv=0.796203
iter=1000, u.uenv=0.996602, v.venv=0.996602
iter=2000, u.uenv=0.996602, v.venv=0.996602
Adiff 0.0739625
A.shape=  (8, 6, 8, 6)

RG step:  1
iter=0, U.Uenv=0.414228
iter=100, U.Uenv=0.414316
iter=200, U.Uenv=0.414317
iter=300, U.Uenv=0.414317
iter=400, U.Uenv=0.414317
error in U, vl, vr optimization: 0.00142528
error in yl, yr, D optimization: 0.0105415
error in w optimization: 0.00745651
iter=0, u.uenv=0.999922, v.venv=0.876962
iter=1000, u.uenv=0.999926, v.venv=0.999926
iter=2000, u.uenv=0.999926, v.venv=0.999926
Adiff 0.0121384
A.shape=  (8, 6, 8, 6)

RG step:  2
iter=0, U.Uenv=0.411154
iter=100, U.Uenv=0.411262
iter=200, U.Uenv=0.411262
iter=300, U.Uenv=0.411262
iter=400, U.Uenv=0.411262
error in U, vl, vr optimization: 0.00158317
error i

In [115]:
i=2
A=A_list[i+1]

In [118]:
T=np.einsum(A,[2,1,2,0])
# ew,_=np.linalg.eigh(T)
ew = scipy.sparse.linalg.eigsh(T, k=100, which='LM', maxiter=200, tol=1e-5, return_eigenvectors=False)
ew.sort()
ew=ew[::-1]
el=-np.log(np.abs(ew/ew[0]))
(el/(el[3]-el[0]))[:100]*0.8

c:\programdata\miniconda3\lib\site-packages\scipy\sparse\linalg\eigen\arpack\arpack.py:1254: RuntimeWarning: k >= N - 1 for N * N square matrix. Attempting to use scipy.linalg.eig instead.
  RuntimeWarning)


array([-0.        ,  0.13235801,  0.13235801,  0.8       ,  0.92473174,
        0.92473174,  1.50283736,  1.33695753,  1.33695752])

In [119]:
T2=np.einsum(A,[4,2,5,0],A,[5,3,4,1]).reshape(chiw**2,chiw**2)
ew,_=np.linalg.eigh(T2)
ew=ew[::-1]
el=-np.log(np.abs(ew/ew[0]))
(el/(el[3]-el[0]))[:100]*0.8

array([-0.        ,  0.13899558,  0.13899558,  0.8       ,  1.10585236,
        1.10585236,  1.11829203,  1.11829203,  1.30798929,  1.30798929,
        1.70819661,  1.70819661,  1.74275149,  1.74976285,  1.757539  ,
        1.78288258,  1.86986997,  1.90832042,  2.04013097,  2.04013097,
        2.08751264,  2.08751264,  2.14380766,  2.14380766,  2.16036015,
        2.16036016,  2.23943557,  2.23943558,  2.27298435,  2.65681528,
        2.70030451,  2.70030451,  2.8101929 ,  2.89160391,  2.90991061,
        3.21370074,  3.21370074,  3.21763855,  3.21763856,  3.27349799,
        3.27349802,  3.40330806,  3.40330807,  3.42702127,  4.12254949,
        4.1225495 ,  5.23935233,  4.90254441,  4.90254438,  4.44936348,
        4.44936347,  4.40196689,  4.36836086,  4.36836076,  4.36182175,
        4.30794043,  4.26860385,  4.26860384,  4.14090674,  3.95558482,
        3.95558479,  3.94649096,  3.92085804,  3.61439828,  3.56224273,
        3.56224273,  3.5530332 ,  3.37189177,  3.37189176,  3.20

In [120]:
T3=np.einsum(A,[10,3,8,0],A,[8,4,9,1],A,[9,5,10,2]).reshape(chiw**3,chiw**3)
ew = scipy.sparse.linalg.eigsh(T3, k=100, which='LM', maxiter=200, tol=1e-5, return_eigenvectors=False)
ew.sort()
ew=ew[::-1]
el=-np.log(np.abs(ew/ew[0]))
(el/(el[3]-el[0]))[:100]*0.8

array([-0.        ,  0.1450817 ,  0.1450817 ,  0.8       ,  1.11114154,
        1.11114154,  1.11114154,  1.11114154,  1.31287496,  1.31287496,
        1.75968898,  1.75968898,  1.80073569,  1.80073569,  1.9777622 ,
        1.9777622 ,  2.07199959,  2.07199959,  2.07199959,  2.07199959,
        2.10863971,  2.10863971,  2.10863971,  2.10863971,  2.12391801,
        2.12391801,  2.3031281 ,  2.3031281 ,  2.3031281 ,  2.3031281 ,
        2.49637849,  2.49637849,  2.65308388,  2.65308388,  2.69259706,
        2.7014563 ,  2.7014563 ,  2.73075741,  2.75339918,  2.75617526,
        2.7851248 ,  2.80345762,  2.91172769,  2.9117277 ,  2.95966914,
        2.95966914,  2.96934782,  2.98511717,  3.00143378,  3.00143378,
        3.00143378,  3.00143378,  3.07042708,  3.07042708,  3.07042708,
        3.07042708,  3.08429546,  3.08429546,  3.09461313,  3.10049875,
        3.10049875,  3.10564777,  3.10564777,  3.10564777,  3.10564777,
        3.29715854,  3.29715854,  3.29715855,  3.29715855,  3.35

In [121]:
T4=np.einsum(A,[11,4,8,0],A,[8,5,9,1],A,[9,6,10,2],A,[10,7,11,3]).reshape(chiw**4,chiw**4)
ew = scipy.sparse.linalg.eigsh(T4, k=100, which='LM', return_eigenvectors=False)
ew.sort()
ew=ew[::-1]
el=-np.log(np.abs(ew/ew[0]))
(el/(el[3]-el[0]))[:100]*0.8

array([-0.        ,  0.15123767,  0.15123767,  0.8       ,  1.1067762 ,
        1.1067762 ,  1.1067762 ,  1.1067762 ,  1.31517811,  1.31517811,
        1.76399162,  1.76399162,  1.79798586,  1.79798586,  1.98112526,
        1.98141536,  2.09161787,  2.09161787,  2.10229574,  2.10229574,
        2.10958357,  2.10958357,  2.12079518,  2.12079518,  2.12684607,
        2.12684607,  2.30919739,  2.30919739,  2.30919739,  2.30919739,
        2.73000568,  2.74928388,  2.76054701,  2.76615308,  2.77563269,
        2.78203967,  2.78882182,  2.8068115 ,  2.94122966,  2.94122966,
        2.96935325,  2.96935325,  2.96935325,  2.96935325,  2.97943426,
        2.97943426,  3.08151929,  3.08151929,  3.08151929,  3.08151929,
        3.09107741,  3.09107741,  3.09107741,  3.09107741,  3.11093913,
        3.11093913,  3.11093913,  3.11093913,  3.13509014,  3.13509014,
        3.13509014,  3.13509014,  3.25779216,  3.25779216,  3.26166576,
        3.26166576,  3.28177134,  3.28177134,  3.29962836,  3.29

In [12]:
T4reduced=np.einsum(A,[11,0,8,0],A,[8,5,9,1],A,[9,6,10,2],A,[10,3,11,3]).reshape(chiw**2,chiw**2)
ew = scipy.sparse.linalg.eigsh(T4reduced, k=100, which='LM', maxiter=200, tol=1e-5, return_eigenvectors=False)
ew.sort()
ew=ew[::-1]
el=-np.log(np.abs(ew/ew[0]))
(el/(el[9]-el[0]))[:100]

c:\programdata\miniconda3\lib\site-packages\scipy\sparse\linalg\eigen\arpack\arpack.py:1254: RuntimeWarning: k >= N - 1 for N * N square matrix. Attempting to use scipy.linalg.eig instead.
  RuntimeWarning)


array([-0.00000000e+00,  4.99596599e-09,  9.20580353e-01,  9.23027805e-01,
        9.74246459e-01,  9.91745214e-01,  9.96888023e-01,  9.97677177e-01,
        9.98760396e-01,  1.00000000e+00,  1.03712251e+00,  1.04902022e+00,
        1.07013854e+00,  1.08046930e+00,  1.12311929e+00,  1.17195892e+00,
        1.18890350e+00,  1.24217920e+00,  1.24406100e+00,  1.27621656e+00,
        1.27699451e+00,  1.28690860e+00,  1.28208019e+00,  1.27945905e+00,
        1.22885842e+00,  1.21879832e+00,  1.19262162e+00,  1.10913150e+00,
        1.10420462e+00,  1.08104846e+00,  1.06700641e+00,  1.05314342e+00,
        9.94678778e-01,  9.41076610e-01,  4.05513452e-01,  4.05513447e-01])

In [92]:
#eval mag
tensor_list={}
tensor_list['A'],tensor_list['U'],tensor_list['vl'],tensor_list['vr'],tensor_list['yl'],tensor_list['yr'],tensor_list['D'],tensor_list['w'],tensor_list['u'],tensor_list['v']=A_list,U_list,vl_list,vr_list,yl_list,yr_list,D_list,w_list,u_list,v_list
op=sz

A0,Az=Asplit(Ainit,chi,op=op)
for i in range(11):
    A0,Az=coarsen(A0,chi,Az)
MOq=MOq_(A0,Az)
eval_op(MOq,tensor_list)  

vertical truncation err: 0.0
horizontal truncation err: 1.0778653232328133e-12
Aout shape: (3, 4, 3, 4)
Coarsen truncation error: 5.37843e-11
Coarsen truncation error: 4.47123e-13
Coarsen truncation error: 1.68353e-11
Coarsen truncation error: 1.71591e-11
Coarsen truncation error: 2.92536e-12
Coarsen truncation error: 5.23275e-10
Coarsen truncation error: 8.30833e-08
Coarsen truncation error: 8.34418e-06
Coarsen truncation error: 0.000354328
Coarsen truncation error: 0.00620844
Coarsen truncation error: 0.0473409
RGsteps: 0 , <O> = 0j
RGsteps: 1 , <O> = (-6.376873039429741e-10+0j)
RGsteps: 2 , <O> = (4.3345908333581516e-10+0j)
RGsteps: 3 , <O> = (3.587927173525184e-09+0j)
RGsteps: 4 , <O> = (1.2692824696695027e-08+0j)
RGsteps: 5 , <O> = (4.910922652640878e-08+0j)
RGsteps: 6 , <O> = (1.9477485328180986e-07+0j)
RGsteps: 7 , <O> = (7.774373633048396e-07+0j)
RGsteps: 8 , <O> = (3.108087404864749e-06+0j)
RGsteps: 9 , <O> = (1.2430687574576725e-05+0j)
RGsteps: 10 , <O> = (4.9721088193973385e

(0.8121454471613788-1.2398831235001642e-16j)

In [70]:
# classical ising:
tensor_list={}
tensor_list['A'],tensor_list['U'],tensor_list['vl'],tensor_list['vr'],tensor_list['yl'],tensor_list['yr'],tensor_list['D'],tensor_list['w'],tensor_list['u'],tensor_list['v']=A_list,U_list,vl_list,vr_list,yl_list,yr_list,D_list,w_list,u_list,v_list
op=sz

A0=A_list[0]
MO=MO_(A0,op)
eval_op(MO,tensor_list)

RGsteps: 0 , <O> = 0j
RGsteps: 1 , <O> = 0j
RGsteps: 2 , <O> = 0j
RGsteps: 3 , <O> = 0j
RGsteps: 4 , <O> = (5.848497230349814e-10+0j)
RGsteps: 5 , <O> = (2.8557120265143097e-09+0j)
RGsteps: 6 , <O> = (1.2696832363946172e-08+0j)
RGsteps: 7 , <O> = (5.30885300837056e-08+0j)
RGsteps: 8 , <O> = (2.136721706764644e-07+0j)
RGsteps: 9 , <O> = (8.561100140332804e-07+0j)
RGsteps: 10 , <O> = (3.4258000281196226e-06+0j)
RGsteps: 11 , <O> = (1.3704560078727831e-05+0j)
RGsteps: 12 , <O> = (5.4819600148251406e-05+0j)
RGsteps: 13 , <O> = (0.0002192797522145205+0j)
RGsteps: 14 , <O> = (0.0008771198348129445+0j)
RGsteps: 15 , <O> = (0.0035084465228825417+0j)
RGsteps: 16 , <O> = (0.014031600599996388+0j)
RGsteps: 17 , <O> = (0.055986888822901086+0j)
RGsteps: 18 , <O> = (0.21545050038725533+0j)
RGsteps: 19 , <O> = (0.560234987406899+0j)
RGsteps: 20 , <O> = (0.6283897009038639+0j)
RGsteps: 21 , <O> = (0.6284032963974049+0j)
RGsteps: 22 , <O> = (0.6284032963974047+0j)
RGsteps: 23 , <O> = (0.628403296397404

(0.6284032963974047+1.040834085829653e-16j)

In [59]:
#exact value
mag_exact=(1-np.sinh(2/T)**(-4))**(1/8)
print(mag_exact)

0.629648162973007


scaling dimensions

In [15]:
i=3
A=A_list[i+1]
Aold,U,vl,vr,yl,yr,D,w,u,v=A_list[i],U_list[i],vl_list[i],vr_list[i],yl_list[i],yr_list[i],D_list[i],w_list[i],u_list[i],v_list[i]


gl=gl_(vl)
gr=gr_(vr)
gu=gu_(Aold,U,vl,vr)
gnw=gnw_(vl,vr,yr,D,w)
gne=gne_(vl,vr,yl,D,w)
gnw,gne=gauge_(gnw,gne,u,v)
chiwt=min(chiw,Aold.shape[1])
chiyt=min(chiy,Aold.shape[0])

def scaleop(v):
    M=v.reshape(chiwt,chiwt,chiyt,chiyt,chiyt,chiyt,chiwt,chiwt)
    Mscaled=rg_(M,gl,gr,gu,gnw,gne)
    return Mscaled.flatten()

n=chiw**4*chiy**4

In [28]:
# compress legs

chik=10
chiq=36
guw,glw,grw,wv,wr,wl = g_compressed(Aold,gu,gr,gl,chik,chiq)

def scaleop(v):
    v4=v.reshape(chiq,chik,chiq,chik)
    out=ncon(v4,[8,9,10,11],glw,[10,19,21],grw,[8,18,20],guw,[9,22,23],guw.conj(),[11,16,17],gnw,[20,22,5,14],gne,[21,23,7,15],gnw.conj(),[18,16,4,12],gne.conj(),[19,17,6,13],wr,[4,5,0],wl,[6,7,2],wv,[14,15,1],wv.conj(),[12,13,3])
    return out.flatten()
n=chik**2*chiq**2

gu compression error: 5.2525290136477224e-05
gr compression error: 0.0
gl compression error: 0.0


In [29]:
import scipy
from scipy.sparse.linalg import LinearOperator

N_level=20
Atemp = LinearOperator((n,n), matvec = scaleop, dtype='complex')
ew = scipy.sparse.linalg.eigs(Atemp, k=N_level, which='LM', maxiter=200, tol=1e-5, return_eigenvectors=False)
ew=ew[::-1]
-np.log2(np.abs(ew/ew[0]))

array([-0.        ,  0.12377507,  1.00079987,  1.12591504,  1.12692008,
        1.99973055,  1.999742  ,  2.0008918 ,  2.00403347,  2.12502254,
        2.12572083,  2.12835146,  2.99483687,  2.99731396,  2.9991284 ,
        3.00452703,  3.01112657,  3.10418136,  3.12282816,  3.12380617])

In [124]:
i=2
A=A_list[i+1]
Aold,U,vl,vr,yl,yr,D,w,u,v=A_list[i],U_list[i],vl_list[i],vr_list[i],yl_list[i],yr_list[i],D_list[i],w_list[i],u_list[i],v_list[i]


wgauged=w@u
# wgauged=w

def scaleop(v):
    M=v.reshape(chiw,chiw,chiw,chiw)
    mera=ncon(M,[0,1,2,3],U,[0,1,8,9],wgauged,[12,8,4],wgauged,[9,13,5],U.conj(),[2,3,10,11],wgauged.conj(),[12,10,6],wgauged.conj(),[11,13,7])
    return mera.flatten()

n=chiw**4

In [125]:
import scipy
from scipy.sparse.linalg import LinearOperator

N_level=25
Atemp = LinearOperator((n,n), matvec = scaleop, dtype='complex')
ew = scipy.sparse.linalg.eigs(Atemp, k=N_level, which='LM', maxiter=200, tol=1e-5, return_eigenvectors=False)
ew=ew[::-1]
-np.log2(np.abs(ew/ew[0]))

array([-0.        ,  0.12811369,  0.12811369,  0.79257194,  0.79257195,
        0.7942325 ,  0.80333101,  0.80333102,  0.94404844,  0.94404844,
        1.08038962,  1.09041317,  1.0984621 ,  1.09935321,  1.22771117,
        1.22771117,  1.23451989,  1.24496526,  1.24496526,  1.24496528,
        1.24496528,  1.2457663 ,  1.24576632,  1.27959849,  1.28089703])

In [127]:
tensor_list['A'],tensor_list['U'],tensor_list['vl'],tensor_list['vr'],tensor_list['yl'],tensor_list['yr'],tensor_list['D'],tensor_list['w'],tensor_list['u'],tensor_list['v']=A_list,U_list,vl_list,vr_list,yl_list,yr_list,D_list,w_list,u_list,v_list

In [129]:
data={'chi':chi,'tensor_list':tensor_list}
np.save('potts',data)